In [6]:
import functions as f
import pandas as pd
import json

In [20]:
PATH = 'realKnownCause/'
ERRORPATH = 'labels/combined_windows.json'
VALUE_OF_CHOICE = 'value_min'

In [21]:
stats_df = pd.DataFrame(columns = ['name', 'time_diff', 'skewness', 'ratio_outlier_nonoutlier', 'ratio_na'])

In [22]:
file_names = f.get_all_files(PATH)
all_csv = [f.read_csv_file(csv) for csv in file_names]
time_diffs, processed_df = list(), list()
for csvfile in all_csv:
    time_diff, df = f.process_time_series(csvfile)
    time_diffs.append(time_diff)
    processed_df.append(df) 

all_error = f.retrieve_error_window(ERRORPATH, file_names)    

In [23]:
stats_rows = []
analyze_dfs = [] 

for i in range(len(processed_df)):
    name = file_names[i].split('/')[-1]
    subdf = processed_df[i]
    suberror_windows = all_error[i]
    respect_time_diff = time_diffs[i]

    # compute metrics
    skewness = f.define_skewness(subdf, VALUE_OF_CHOICE)
    ratio_na =  f.ratio_na(subdf, VALUE_OF_CHOICE)
    # if there is null value, fill with linear interpolation
    if f.ratio_na(subdf, VALUE_OF_CHOICE) != 0:        
        subdf[VALUE_OF_CHOICE] = subdf[VALUE_OF_CHOICE].interpolate(method='linear') 
    # define outlier 
    subdf['outlier'] = subdf['timestamp'].apply(lambda x: f.is_outlier(x, suberror_windows))
    ratio_outlier = f.ratio_outlier(subdf)
    
    new_row = {
        'name': name,
        'time_diff': respect_time_diff,
        'skewness': skewness,
        'ratio_outlier_nonoutlier': ratio_outlier,
        'ratio_na': ratio_na
    }
    stats_rows.append(new_row) 
    
    periods = f.period_pattern(subdf, respect_time_diff)
    character_df = f.analyze_time_series(subdf, periods, VALUE_OF_CHOICE)
    character_df['name'] = [name]*len(character_df)
    analyze_dfs.append(character_df) 

# Convert lists to DataFrames
stats_df = pd.DataFrame(stats_rows)
character_df_combined = pd.concat(analyze_dfs, ignore_index=True)

# Merge the final stats and character data on 'name'
final_df = pd.merge(character_df_combined, stats_df, on='name', how='left')

/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 10272.
  res = hypotest_fun_out(*samples, **kwds)
/projappl/project_2012638/thesis/functions.py:170: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results_df = pd.concat([results_df, new_row], ignore_index=True)
/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/scipy/stats/_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: For N > 5000, computed p-value may not be accurate. Current N is 9984.
  res = hypotest_fun_out(*samples, **kwds)
/PUHTI_TYKKY_FRQGCcR/miniconda/envs/env1/lib/python3.10/site-packages/scipy/s

In [24]:
final_df

,period_name,trend_var,seasonal_var,residual_var,residual_normality,trend,trend_slope,significant_seasonal,name,time_diff,skewness,ratio_outlier_nonoutlier,ratio_na
0,perday,4.718942e+06,3.181859e+07,1.169710e+07,False,no trend,-0.011565,True,nyc_taxi.csv,1800.0,negative,0.111470,0.00
1,perweek,1.189376e+06,4.252862e+07,3.818240e+06,False,no trend,-0.011565,True,nyc_taxi.csv,1800.0,negative,0.111470,0.00
2,permonth,4.048633e+05,3.334405e+07,1.351207e+07,False,no trend,-0.011565,True,nyc_taxi.csv,1800.0,negative,0.111470,0.00
3,perday,1.237502e-03,1.957627e-04,1.790005e-03,False,increasing,0.000002,True,rogue_agent_key_hold.csv,300.0,positive,0.082539,1.84
4,perweek,1.935583e-04,1.136296e-03,7.575181e-04,False,increasing,0.000002,True,rogue_agent_key_hold.csv,300.0,positive,0.082539,1.84
5,perday,1.893887e+01,1.585785e+02,6.250419e+01,False,decreasing,-0.000039,True,cpu_utilization_asg_misconfiguration.csv,300.0,positive,0.090503,0.00
6,perweek,4.140099e+00,1.781909e+02,5.169338e+01,False,decreasing,-0.000039,True,cpu_utilization_asg_misconfiguration.csv,300.0,positive,0.090503,0.00
7,permonth,3.221172e-01,1.860196e+02,2.470714e+00,False,decreasing,-0.000039,True,cpu_utilization_asg_misconfiguration.csv,300.0,positive,0.090503,0.00
8,perday,1.521260e+01,5.036648e-01,1.136564e+00,False,decreasing,-0.000552,True,ambient_temperature_system_failure.csv,3600.0,negative,0.128631,0.09
9,perweek,1.210951e+01,1.714584e+00,2.816079e+00,False,decreasing,-0.000552,True,ambient_temperature_system_failure.csv,3600.0,negative,0.128631,0.09


In [13]:
processed_df[1]

,timestamp,value_min,value_max,value_mean,outlier
0,2014-07-06 19:15:00,0.063880,0.065692,0.064600,0
1,2014-07-06 20:32:00,0.000000,0.067751,0.031565,0
2,2014-07-06 21:49:00,NaN,NaN,NaN,0
3,2014-07-06 23:06:00,NaN,NaN,NaN,0
4,2014-07-07 00:23:00,NaN,NaN,NaN,0
...,...,...,...,...,...
343,2014-07-25 03:26:00,0.066957,0.068055,0.067385,0
344,2014-07-25 04:43:00,0.066791,0.092549,0.075977,0
345,2014-07-25 06:00:00,0.000000,0.321493,0.086451,0
346,2014-07-25 07:17:00,0.000000,0.099511,0.042077,0


In [15]:
time_diffs[1]

4620.000000000027